## IMPORT

In [5]:
import sys
import os
from pathlib import Path
import numpy as np
from shock import shock
import _mutationpp as mpp
import rebuilding_setup as setup
import reading_input as input_data
input_dict = input_data.reading_input()
from forward import forward

#To run locally and find the modules in /src
cabaret_src_folder = '/Users/jeannelonglune/Desktop/memoire/Mutationpp-master/src'
sys.path.insert(0, cabaret_src_folder)

mutation_folder = "/Users/jeannelonglune/Desktop/memoire/Mutationpp-master/" 
my_distribution = "macosx-13.0-arm64-3.11" 
sys.path.append(mutation_folder + "_skbuild/" + my_distribution +"/cmake-install/interface/python/mutationpp")

## Importing local mutationpp python module ##
mppPyDir=os.environ.get('MPP_LOCALPY')
sys.path.append(mppPyDir)

## Set up

In [9]:
opts = mpp.MixtureOptions("N")
mix = mpp.Mixture(opts)
set_state_with_rhoi_T = 1
pos_T_trans = 0
ns = mix.nSpecies()
nT = mix.nEnergyEqns()

# Define gas conditions (T = 6000 K and p = 1000 Pa)
Teq = np.ones(nT) * 6000 #K
Peq = 1000.  # Pa

## Equilibrium conditions

In [10]:
mix.equilibrate(Teq[pos_T_trans], Peq)

In [6]:
preshock_state = [input_dict["freestream"]["Temperature"],input_dict["freestream"]["Pressure"],input_dict["freestream"]["Mach"]]
output = forward(preshock_state,input_dict["residual"],input_dict["throat_area"],input_dict["effective_radius"],input_dict["surface_temperature"],input_dict["Prandtl"],input_dict["Lewis"],mix,input_dict["print_info"],input_dict["options"])

print('-----OUTPUT------', output)
"""preshock_state : 1D array of size 2
        Temperature and pressure of the pre shock state."""
setup.mixture_states(mix)["free_stream"].equilibrate(preshock_state[0],preshock_state[1])
rhoi = setup.mixture_states(mix)["free_stream"].density()

[243.0, 18.53, 26.3]
-----OUTPUT------ {'Reservoir_temperature': 17233.04789796878, 'Reservoir_pressure': 35122393270.80414, 'Mass_flow': 29870.938317124146, 'Total_enthalpy': 33845535.57175973, 'Heat_flux': 2619194.674302837, 'Stagnation_density': 0.004776289245203909, 'Stagnation_pressure': 17455.59192367097, 'Free_stream_density': 0.00026459726067635674, 'Free_stream_velocity': 8234.224505024145, 'Free_stream_pressure': 18.53}


In [11]:
Tg = output["Reservoir_temperature"]  #check what is Tg 
#print(type(Tg))
#print(type(rhoi))
#print(type(set_state_with_rhoi_T))
mix.setState(rhoi, Tg, set_state_with_rhoi_T)

T_s = np.ones(nT) * 6000
#T_s = VectorXd.Constant(nT, 6000.)  # change 6000 to the right value (surface temperature)

# Mass gradient
xi_e = np.array(mix.X())
dx = 1e-2  # to be changed to the right value

# see Newton method, tentative value
rhoi_s = xi_e.copy()  # equal to gas value

#classe de mix est une classe fille et la fonciton ici est def de la classe mère et non de la classe fille. 
# Solving mass balance

#import cwrapper
print(rhoi_s)
print(type(rhoi_s))
rhoi_s = [1.0, 3.0, 3.0]
#rhoi_s = np.array(rhoi_s, dtype=float)
print(type(rhoi_s[0]))
T_s = [1.2, 3.2, 3.1]
print(T_s)
set_state_with_rhoi_T = 1
print(set_state_with_rhoi_T)

[7.45432334e-009 9.99999993e-001 0.00000000e+000 1.46443034e-318]
<class 'numpy.ndarray'>
<class 'float'>
[1.2, 3.2, 3.1]
1
